In [8]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [9]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.core import Environment, Workspace
from dotenv import load_dotenv
import os
from azureml.core.compute import ComputeTarget
# Load .env file
load_dotenv()

from azureml.core import Environment
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Initialize Azure ML Workspace
ws = Workspace.from_config()

In [10]:

# Create the environment from requirements.txt
python_env = Environment.from_pip_requirements("pipeline_env", "requirements.txt")

# Set environment variables
env_vars = {
    "subscription_id": os.getenv("subscription_id"),
    "resource_group": os.getenv("resource_group"),
    "workspace_name": os.getenv("workspace_name"),
    "tenant_id": os.getenv("tenant_id"),
    "client_id": os.getenv("client_id"),
    "client_secret": os.getenv("client_secret"),
}

python_env.environment_variables = env_vars

Property environment_variables is deprecated. Use RunConfiguration.environment_variables to set runtime variables.


In [13]:
# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()
pipeline_run_config.environment = python_env

# Define the name of your existing compute target
existing_compute_target_name = "matthewhanauer993"

# Retrieve the existing compute target
compute_target = ComputeTarget(workspace=ws, name=existing_compute_target_name)

# Validate that the compute target exists
if compute_target is not None:
    print(f"Using existing compute target: {existing_compute_target_name}")
else:
    print(f"No compute target found with name {existing_compute_target_name}")
    exit(1)  # Exit the script if the compute target doesn't exist

data_loading_step = PythonScriptStep(
    name="Data Loading Step",
    script_name="data_loading.py",
    source_directory="./",
    runconfig=pipeline_run_config,
    compute_target=compute_target,
    allow_reuse=False,
)

analysis_step = PythonScriptStep(
    name="Analysis Step",
    script_name="analysis.py",
    source_directory="./",
    runconfig=pipeline_run_config,
    compute_target=compute_target,
    allow_reuse=False,
)

analysis_step.run_after(data_loading_step)

# Create and submit the pipeline
# Create a Pipeline
pipeline = Pipeline(workspace=ws, steps=[data_loading_step, analysis_step])
# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name="My_Pipeline")
pipeline_run = experiment.submit(pipeline)

experiment_name = "My_Docker_Pipeline"
experiment = Experiment(workspace=ws, name=experiment_name)
pipeline_run = experiment.submit(pipeline)

published_pipeline = pipeline.publish(
    name="My_Published_Pipeline",
    description="Published pipeline to run the script every minute.",
)

pipeline_id = published_pipeline.id

Using existing compute target: matthewhanauer993
Created step Data Loading Step [b8259a70][d1d2f25f-314a-4e1b-85e5-25617e72baea], (This step will run and generate new outputs)
Created step Analysis Step [40bfc734][b2472af3-a7d5-4aff-9479-322c4218f556], (This step will run and generate new outputs)
Submitted PipelineRun 0a3e687b-98e9-4422-a1fa-1079a85d925f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0a3e687b-98e9-4422-a1fa-1079a85d925f?wsid=/subscriptions/d3ade6e0-a7be-4323-ace4-02b7b4da2451/resourcegroups/ml_modeling/workspaces/mhanauer_1&tid=a93eafc8-c064-4a88-bc01-47e93068b42b
Submitted PipelineRun c48648a6-b98d-4182-a65b-aab8e9745d89
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c48648a6-b98d-4182-a65b-aab8e9745d89?wsid=/subscriptions/d3ade6e0-a7be-4323-ace4-02b7b4da2451/resourcegroups/ml_modeling/workspaces/mhanauer_1&tid=a93eafc8-c064-4a88-bc01-47e93068b42b


In [14]:
# Define the pipeline schedule
recurrence = ScheduleRecurrence(frequency="Minute", interval=1)  # Runs every minute
schedule = Schedule.create(
    ws,
    name="My_Schedule",
    description="Runs pipeline every minute",
    pipeline_id=pipeline_id,
    experiment_name=experiment_name,
    recurrence=recurrence,
)

In [15]:
# Your schedule name
schedule_name = "My_Schedule"

# List all schedules in the workspace
schedules = Schedule.list(ws)

# Filter and disable the schedule with the given name
for schedule in schedules:
    if schedule.name == schedule_name:
        print(f"Disabling schedule {schedule.name}")
        schedule.disable()

Disabling schedule My_Schedule
